# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

# Load Dataset

In [196]:
def preprocess_data(filename: str):
    with open(filename, mode="r") as f:
        data = f.read()
        first_place = data.find("{")
        data = data[first_place:]

        lines = data.split('\n')

        lines = lines[2:]

        training_set = []

        for line in lines[:-1]:
            input = {}

            line = line[1:-1]

            data = line.split(',')

            for feature in data[:-1]:
                key, time = feature.strip().split(' ')

                input[int(key)] = int(time)
            # print(data[-1].strip()[4:])
            input['spam'] = int(data[-1].strip()[4:].strip() == 'spam')

            training_set.append(input)

        return training_set

In [227]:
training_set = preprocess_data(filename='./q4/spam_train_2000.arff')
print(len(training_set))

testing_set = preprocess_data(filename='./q4/spam_test.arff')
print(len(testing_set))

2000
883


In [235]:
train_size = [10, 25, 50, 100, 200, 300, 400, 500, 750, 1000, 1250, 1500, 1750, 2000]
training_set = []
for size in train_size:
    training_set.extend(preprocess_data(filename=f'./q4/spam_train_{size}.arff'))
print(len(training_set))

testing_set = preprocess_data(filename='./q4/spam_test.arff')
print(len(testing_set))

9801
883


In [236]:
# Let's build some naive bayes

# We assume that y ~ Bern(\phi)
# x|y = 0 ~ N(\u_o, \sigma)
# x|y = 1 ~ N(\u_1, \sigma)
import numpy as np

PROB_DICT = {}      
WORD_NON_SPAM_PROB = np.array([1]*1446)
WORD_SPAM_PROB = np.array([1]*1446)

In [237]:
# Count \phi_y
SPAM = 0
ALL_WORD_SPAM = 1446
ALL_WORD_NON_SPAM = 1446

for x in training_set:
    if x['spam'] == 1:
        SPAM += 1

        for key in x:
            if type(key) is int:
                time = x[key]

                ALL_WORD_SPAM += time
                if key < 1447:
                    WORD_SPAM_PROB[key - 1] += time

    else:
        for key in x:
            if type(key) is int:
                time = x[key]

                ALL_WORD_NON_SPAM += time
                
                if key < 1447:
                    WORD_NON_SPAM_PROB[key - 1] += time

In [238]:
PROB_DICT['spam'] = SPAM / len(training_set)
WORD_SPAM_PROB = WORD_SPAM_PROB * 100 / ALL_WORD_SPAM
WORD_NON_SPAM_PROB = WORD_NON_SPAM_PROB * 100 / ALL_WORD_NON_SPAM

In [239]:
print(PROB_DICT)
print(WORD_SPAM_PROB)
print(WORD_NON_SPAM_PROB)
print(ALL_WORD_SPAM)
print(ALL_WORD_NON_SPAM)

{'spam': 0.506478930721355}
[0.0540815  0.05843288 0.08329794 ... 0.02424343 0.01906321 0.19581231]
[0.08593315 0.06474416 0.07298432 ... 0.06180124 0.01824608 0.11447944]
482605
339799


In [240]:
# TESTING ON THE TESTING SET

correct = 0
size = len(testing_set)

for i, x_new in enumerate(testing_set):
    spam_prob = PROB_DICT['spam']
    non_spam_prob = 1 - spam_prob

    for key in x_new:
        if type(key) is int:
            spam_prob *= WORD_SPAM_PROB[key - 1]

            non_spam_prob *= WORD_NON_SPAM_PROB[key -1]
    
    predict_label = int(spam_prob >= non_spam_prob)

    # print(f'{i}. Spam: {spam_prob}, Non-spam: {non_spam_prob}, predict: {predict_label} - {x_new['spam']}')

    if predict_label == x_new['spam']:
        correct += 1
    else:
        print(f'{i}. Spam: {spam_prob}, Non-spam: {non_spam_prob}, predict: {predict_label} - {x_new['spam']}')


2. Spam: 3.431104260469514e-51, Non-spam: 2.9979171044112425e-53, predict: 1 - 0
27. Spam: 1.739585553443896e-52, Non-spam: 5.717728442639841e-53, predict: 1 - 0
91. Spam: 5.930315827359944e-30, Non-spam: 2.299323574029138e-30, predict: 1 - 0
100. Spam: 8.546530995326098e-41, Non-spam: 3.5530299662954453e-40, predict: 0 - 1
109. Spam: 2.213341691775929e-36, Non-spam: 2.3901664723426177e-35, predict: 0 - 1
117. Spam: 8.64792632433141e-31, Non-spam: 5.163761596520059e-30, predict: 0 - 1
118. Spam: 2.2341075150481433e-60, Non-spam: 2.2845603726500283e-60, predict: 0 - 1
128. Spam: 8.923635085504642e-129, Non-spam: 1.467867836770509e-128, predict: 0 - 1
150. Spam: 3.8214504262448424e-22, Non-spam: 8.847992595386407e-22, predict: 0 - 1
151. Spam: 0.18237442606641557, Non-spam: 0.451762048319356, predict: 0 - 1
158. Spam: 2.1022248810392577e-111, Non-spam: 4.187081872719896e-108, predict: 0 - 1
173. Spam: 1.7628778166698023e-71, Non-spam: 3.256141292681904e-70, predict: 0 - 1
183. Spam: 8.94

In [241]:
# Lets find out the accuracy
print(f'Accuracy: {correct} / {size}')

Accuracy: 815 / 883
